In [ ]:
import os
import yaml
import pandas as pd
from tqdm import tqdm

DATA_DIR = 'D:\Mainboot Project\stock\data'
OUTPUT_DIR = 'output1_csv'
os.makedirs(OUTPUT_DIR, exist_ok=True)

date_range = pd.date_range(start='2023-10-01', end='2024-11-30', freq='B')  

symbol_data = {}

for dt in tqdm(date_range, desc="Processing Dates"):
    try:
        month_folder = dt.strftime("%Y-%m")
        date_str = dt.strftime("%Y-%m-%d_%H-%M-%S")
        file_name = f"{dt.strftime('%Y-%m-%d')}_05-30-00.yaml"
        file_path = os.path.join(DATA_DIR, month_folder, file_name)

        if not os.path.exists(file_path):
            continue

        with open(file_path, 'r') as file:
            data = yaml.safe_load(file)
            if not data:
                continue

            for record in data:
                symbol = record.get('Ticker')
                if not symbol:
                    continue

                if symbol not in symbol_data:
                    symbol_data[symbol] = []

                symbol_data[symbol].append({
                    'date': record.get('date'),
                    'open': record.get('open'),
                    'high': record.get('high'),
                    'low': record.get('low'),
                    'close': record.get('close'),
                    'volume': record.get('volume'),
                    'month': record.get('month')
                })
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Save CSV files
for symbol, records in symbol_data.items():
    df = pd.DataFrame(records)
    df.sort_values(by='date', inplace=True)
    csv_path = os.path.join(OUTPUT_DIR, f"{symbol}.csv")
    df.to_csv(csv_path, index=False)

print("All symbol-wise CSV files saved.")


<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:7: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Sriraman\AppData\Local\Temp\ipykernel_24708\4017541062.py:7: SyntaxWarning: invalid escape sequence '\M'
  DATA_DIR = 'D:\Mainboot Project\stock\data'
Processing Dates: 100%|██████████| 305/305 [00:19<00:00, 15.79it/s]


✅ All symbol-wise CSV files saved.


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

INPUT_DIR = 'output1_csv'
CLEANED_DIR = 'cleaned_csv'
os.makedirs(CLEANED_DIR, exist_ok=True)

csv_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.csv')]

for file in tqdm(csv_files, desc="Cleaning CSVs"):
    try:
        file_path = os.path.join(INPUT_DIR, file)
        df = pd.read_csv(file_path)

        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['open'] = pd.to_numeric(df['open'], errors='coerce')
        df['high'] = pd.to_numeric(df['high'], errors='coerce')
        df['low'] = pd.to_numeric(df['low'], errors='coerce')
        df['close'] = pd.to_numeric(df['close'], errors='coerce')
        df['volume'] = pd.to_numeric(df['volume'], errors='coerce')

        if 'month' not in df.columns or df['month'].isnull().all():
            df['month'] = df['date'].dt.to_period('M').astype(str)  # Fallback if month is missing
        else:
            df['month'] = df['month'].astype(str)

        df.dropna(inplace=True)
        df.sort_values(by='date', inplace=True)
        df.drop_duplicates(subset=['date'], keep='last', inplace=True)
        cleaned_path = os.path.join(CLEANED_DIR, file)
        df.to_csv(cleaned_path, index=False)

    except Exception as e:
        print(f"Error cleaning {file}: {e}")

print("All cleaned CSV files saved with 'month' column.")



Cleaning CSVs: 100%|██████████| 50/50 [00:01<00:00, 25.35it/s]

✅ All cleaned CSV files saved with 'month' column.


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

CLEANED_DIR = 'cleaned_csv'
COMBINED_CSV = 'combined_stocks.csv'

all_data = []

csv_files = [f for f in os.listdir(CLEANED_DIR) if f.endswith('.csv')]

for file in tqdm(csv_files, desc="Combining CSVs"):
    file_path = os.path.join(CLEANED_DIR, file)
    df = pd.read_csv(file_path)
    symbol = os.path.splitext(file)[0]
    df['symbol'] = symbol

    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)
combined_df.sort_values(by=['date', 'symbol'], inplace=True)
combined_df.to_csv(COMBINED_CSV, index=False)

print(f"Combined CSV saved as: {COMBINED_CSV}")


Combining CSVs: 100%|██████████| 50/50 [00:01<00:00, 33.64it/s]


✅ Combined CSV saved as: combined_stocks.csv


In [ ]:
# 1. Yearly Returns, Top Gainers & Losers, Market Summary
import pandas as pd

df = pd.read_csv('combined_stocks.csv')
df['date'] = pd.to_datetime(df['date'])
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)

first_close = df.sort_values(by='date').groupby('symbol').first()['close']
last_close = df.sort_values(by='date').groupby('symbol').last()['close']

yearly_return_df = pd.DataFrame({
    'symbol': first_close.index,
    'first_close': first_close.values,
    'last_close': last_close.values,
    'yearly_return': ((last_close.values - first_close.values) / first_close.values) * 100
})
yearly_return_df.to_csv('yearly_returns.csv', index=False)

# Top 10 Green & Loss Stocks
yearly_return_df.sort_values(by='yearly_return', ascending=False).head(10).to_csv('top_10_green_stocks.csv', index=False)
yearly_return_df.sort_values(by='yearly_return', ascending=True).head(10).to_csv('top_10_loss_stocks.csv', index=False)

# Market Summary
market_summary_df = pd.DataFrame([{
    'green_stocks': (yearly_return_df['yearly_return'] > 0).sum(),
    'red_stocks': (yearly_return_df['yearly_return'] <= 0).sum(),
    'avg_close_price': round(df['close'].mean(), 2),
    'avg_volume': int(df['volume'].mean())
}])
market_summary_df.to_csv('market_summary.csv', index=False)


In [ ]:
# 2. Volatility (Top 10 Volatile Stocks)

df['daily_return'] = df.groupby('symbol')['close'].pct_change()
volatility = df.groupby('symbol')['daily_return'].std().reset_index()
volatility.columns = ['symbol', 'volatility']
volatility.dropna(inplace=True)
volatility.sort_values(by='volatility', ascending=False).head(10).to_csv('top_10_volatile_stocks.csv', index=False)

In [ ]:
# 3. Cumulative Return by Date (Top 5 Stocks)

df['daily_return'] = df.groupby('symbol')['close'].pct_change().fillna(0)
df['cumulative_return'] = df.groupby('symbol')['daily_return'].transform(lambda x: (1 + x).cumprod() - 1)

final_returns = df.groupby('symbol')['cumulative_return'].last().reset_index(name='final_cumulative_return')
top_5_symbols = final_returns.sort_values(by='final_cumulative_return', ascending=False).head(5)['symbol']

top_5_df = df[df['symbol'].isin(top_5_symbols)]
top_5_df['date'] = top_5_df['date'].dt.date
top_5_df[['date', 'symbol', 'cumulative_return']].to_csv('cumulative_returns_by_date.csv', index=False)

C:\Users\Sriraman\AppData\Local\Temp\ipykernel_16884\3730455587.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_df['date'] = top_5_df['date'].dt.date


In [ ]:
# 4. Sector-wise Performance
returns = df.groupby('symbol').agg(
    first_close=('close', 'first'),
    last_close=('close', 'last')
).reset_index()
returns['cumulative_return'] = (returns['last_close'] - returns['first_close']) / returns['first_close']

sector_df = pd.read_csv("Sector_data - Sheet1.csv")
sector_df['symbol'] = sector_df['Symbol'].str.split(':').str[-1].str.strip()

returns = pd.merge(returns, sector_df[['sector', 'symbol']], on='symbol', how='left')
returns.dropna(subset=['sector'], inplace=True)

returns.groupby('sector')['cumulative_return'].mean().reset_index(name='average_return')\
    .sort_values(by='average_return', ascending=False).to_csv("sector_wise_performance.csv", index=False)


In [ ]:
# 5. Stock Correlation
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['symbol', 'date'], inplace=True)

df['daily_return'] = df.groupby('symbol')['close'].pct_change()
pivot_df = df.pivot(index='date', columns='symbol', values='daily_return')

correlation_matrix = pivot_df.corr()
correlation_matrix.to_csv('correlation_matrix.csv')  

flat_corr = correlation_matrix.unstack().rename_axis(['Stock_1', 'Stock_2']).reset_index(name='correlation')
flat_corr = flat_corr[flat_corr['Stock_1'] != flat_corr['Stock_2']]
flat_corr['pair'] = flat_corr.apply(lambda x: tuple(sorted([x['Stock_1'], x['Stock_2']])), axis=1)
flat_corr = flat_corr.drop_duplicates('pair').drop(columns='pair')

flat_corr.to_csv('stock_price_correlation_pairs.csv', index=False)

print("All CSVs created successfully.")


All CSVs created successfully.


In [ ]:
# 6. Monthly Top 5 Gainers & Losers
df['month'] = df['date'].dt.to_period('M')
monthly_returns = df.groupby(['symbol', 'month'])['close'].agg(['first', 'last']).reset_index()
monthly_returns['monthly_return'] = (monthly_returns['last'] - monthly_returns['first']) / monthly_returns['first']

top5_gainers, top5_losers = [], []
for month in monthly_returns['month'].unique():
    data = monthly_returns[monthly_returns['month'] == month]
    top5_gainers.append(data.nlargest(5, 'monthly_return').assign(rank=range(1, 6), month=month))
    top5_losers.append(data.nsmallest(5, 'monthly_return').assign(rank=range(1, 6), month=month))

gainers_df = pd.concat(top5_gainers)[['month', 'symbol', 'monthly_return', 'rank']]
losers_df = pd.concat(top5_losers)[['month', 'symbol', 'monthly_return', 'rank']]

gainers_df.to_csv("monthly_top5_gainers.csv", index=False)
losers_df.to_csv("monthly_top5_losers.csv", index=False)


In [ ]:
import mysql.connector

initial_conn = mysql.connector.connect(
    host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',
    user='2Djg5GyoVwWC4gN.root',
    password='JWYIDlQjiVFU2pUD',
    port=4000
)
initial_cursor = initial_conn.cursor()

initial_cursor.execute("CREATE DATABASE IF NOT EXISTS stocks_analysis")
print("atabase 'stocks_analysis' created or already exists.")
initial_cursor.close()
initial_conn.close()

✅ Database 'stocks_analysis' created or already exists.


In [ ]:
import os
import pandas as pd
import mysql.connector

conn = mysql.connector.connect(
    host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',       
    user='2Djg5GyoVwWC4gN.root',
    password='JWYIDlQjiVFU2pUD',
    database='stocks_analysis',
    port=4000                    
)
cursor = conn.cursor()

csv_files = [
    "top_10_volatile_stocks.csv",
    "yearly_returns.csv",
    "top_10_loss_stocks.csv",
    "top_10_green_stocks.csv",
    "stock_price_correlation_pairs.csv",
    "sector_wise_performance.csv",
    "monthly_top5_gainers.csv",
    "monthly_top5_losers.csv",
    "market_summary.csv",
    "cumulative_returns_by_date.csv",
    "correlation_matrix.csv"
    
]

for csv_file in csv_files:
    if not os.path.exists(csv_file):
        print(f"File not found: {csv_file}")
        continue

    table_name = os.path.splitext(os.path.basename(csv_file))[0].lower()
    df = pd.read_csv(csv_file)

    cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")

    columns = []
    for col, dtype in zip(df.columns, df.dtypes):
        if 'int' in str(dtype):
            sql_type = 'INT'
        elif 'float' in str(dtype):
            sql_type = 'FLOAT'
        elif 'datetime' in str(dtype):
            sql_type = 'DATETIME'
        else:
            sql_type = 'VARCHAR(255)'
        columns.append(f"`{col}` {sql_type}")
    create_query = f"CREATE TABLE `{table_name}` ({', '.join(columns)})"
    cursor.execute(create_query)

    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else str(x) for x in row)
        placeholders = ', '.join(['%s'] * len(values))
        insert_query = f"INSERT INTO `{table_name}` VALUES ({placeholders})"
        cursor.execute(insert_query, values)

    conn.commit()
    print(f"Loaded `{csv_file}` into `{table_name}`")

cursor.close()
conn.close()
print("All CSVs loaded into TiDB successfully.")


✅ Loaded `top_10_volatile_stocks.csv` into `top_10_volatile_stocks`
✅ Loaded `yearly_returns.csv` into `yearly_returns`
✅ Loaded `top_10_loss_stocks.csv` into `top_10_loss_stocks`
✅ Loaded `top_10_green_stocks.csv` into `top_10_green_stocks`
✅ Loaded `stock_price_correlation_pairs.csv` into `stock_price_correlation_pairs`
✅ Loaded `sector_wise_performance.csv` into `sector_wise_performance`
✅ Loaded `monthly_top5_gainers.csv` into `monthly_top5_gainers`
✅ Loaded `monthly_top5_losers.csv` into `monthly_top5_losers`
✅ Loaded `market_summary.csv` into `market_summary`
✅ Loaded `cumulative_returns_by_date.csv` into `cumulative_returns_by_date`
✅ Loaded `correlation_matrix.csv` into `correlation_matrix`
🎉 All CSVs loaded into TiDB successfully.
